<a href="https://colab.research.google.com/github/werowe/HypatiaAcademy/blob/master/stats/consolidate_weather_data_run_sql_spark_queries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import pandas as pd

# Define the directory containing the CSV files
directory = 'drive/MyDrive/weather'

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with 'paphos2024' and ends with '.csv'
    if filename.startswith('paphos20') and filename.endswith('.csv'):
        # Read the CSV file into a DataFrame
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        dataframes.append(df)

# Combine all DataFrames into one
df_combined = pd.concat(dataframes, ignore_index=True)

# Drop duplicate rows
df_combined = df_combined.drop_duplicates()

# Save the combined DataFrame to a new CSV file (optional)
df_combined.to_csv('combined_weather.csv', index=False)

# Print a summary of the combined DataFrame
print(f"Combined DataFrame shape: {df_combined.shape}")

Combined DataFrame shape: (30751, 24)


In [3]:
from pyspark.sql import SparkSession


# Initialize SparkSession
spark = SparkSession.builder \
    .appName("weather") \
    .getOrCreate()

df = spark.read.csv(
    "combined_weather.csv",
    header=True,        # Use the first row as column names
    inferSchema=True,   # Automatically infer data types
    sep=",",            # Specify delimiter (default is ',')
    encoding="UTF-8"    # Handle encoding
)



In [4]:
df.columns

['name',
 'datetime',
 'temp',
 'feelslike',
 'dew',
 'humidity',
 'precip',
 'precipprob',
 'preciptype',
 'snow',
 'snowdepth',
 'windgust',
 'windspeed',
 'winddir',
 'sealevelpressure',
 'cloudcover',
 'visibility',
 'solarradiation',
 'solarenergy',
 'uvindex',
 'severerisk',
 'conditions',
 'icon',
 'stations']

In [5]:
df.createOrReplaceTempView("weather")

In [6]:

sql = '''
SELECT round(SUM(precip)) AS total_precip, YEAR(datetime) AS year, MONTH(datetime) AS month
FROM weather
GROUP BY YEAR(datetime), MONTH(datetime)
ORDER BY year, month

'''

result = spark.sql(sql)
result.show()

+------------+----+-----+
|total_precip|year|month|
+------------+----+-----+
|        69.0|2024|    1|
|        35.0|2024|    2|
|         9.0|2024|    3|
|       239.0|2024|    4|
|        18.0|2024|    5|
|         0.0|2024|    6|
|         0.0|2024|    7|
|         0.0|2024|    8|
|         2.0|2024|    9|
|         2.0|2024|   10|
|       166.0|2024|   11|
|       487.0|2024|   12|
|        64.0|2025|    1|
|       108.0|2025|    2|
|        45.0|2025|    3|
|        84.0|2025|    4|
|        58.0|2025|    5|
|         3.0|2025|    6|
|         2.0|2025|    7|
|         0.0|2025|    8|
+------------+----+-----+



In [7]:
# is it cloudier this year than last


sql = '''
SELECT CAST(SUM(cloudcover) AS INT) AS clouds, YEAR(datetime) AS year, MONTH(datetime) AS month
FROM weather
GROUP BY YEAR(datetime), MONTH(datetime)
HAVING MONTH(datetime) = 5
ORDER BY year, month

'''


result = spark.sql(sql)
result.show()

+------+----+-----+
|clouds|year|month|
+------+----+-----+
| 28069|2024|    5|
|103483|2025|    5|
+------+----+-----+



In [8]:
# is it rainier this year than last


sql = '''
SELECT CAST(SUM(precip) AS INT) AS precip, YEAR(datetime) AS year, MONTH(datetime) AS month
FROM weather
GROUP BY YEAR(datetime), MONTH(datetime)
HAVING MONTH(datetime) = 5
ORDER BY year, month

'''


result = spark.sql(sql)
result.show()

+------+----+-----+
|precip|year|month|
+------+----+-----+
|    18|2024|    5|
|    57|2025|    5|
+------+----+-----+



In [9]:
# maximum temperature


sql = '''
SELECT round(max(temp),2) AS max_temp, date_format(datetime, 'yy-MM-dd') as YYMMDD
FROM weather
GROUP BY YYMMDD
ORDER BY max_temp desc

'''

result = spark.sql(sql)
result.show()

+--------+--------+
|max_temp|  YYMMDD|
+--------+--------+
|    37.6|24-06-07|
|    35.8|25-08-14|
|    35.5|25-08-13|
|    34.9|24-06-08|
|    34.8|24-06-13|
|    34.6|24-07-28|
|    33.8|25-07-27|
|    33.5|24-06-06|
|    33.5|25-08-12|
|    33.4|24-07-29|
|    33.4|25-08-09|
|    33.4|25-07-26|
|    33.3|24-04-24|
|    33.3|24-07-20|
|    33.3|24-07-27|
|    33.3|24-07-21|
|    33.2|24-06-14|
|    33.2|25-07-25|
|    33.1|24-08-17|
|    33.0|24-08-01|
+--------+--------+
only showing top 20 rows



# Question:  When does heat drop off in the fall

In [21]:
sql = '''
SELECT round(avg(temp),2) AS ave_temp, round(max(temp),2) AS max_temp,  MONTH(datetime) AS month
FROM weather

GROUP BY MONTH(datetime)
HAVING MONTH(datetime) in (8, 9, 10, 11)
ORDER BY month

'''

result = spark.sql(sql)
result.show()

+--------+--------+-----+
|ave_temp|max_temp|month|
+--------+--------+-----+
|   28.55|    35.8|    8|
|   26.56|    32.1|    9|
|   22.68|    28.8|   10|
|   18.29|    27.0|   11|
+--------+--------+-----+



In [22]:
sql = '''
SELECT round(avg(temp),2) AS ave_temp, round(max(temp),2) AS max_temp,  MONTH(datetime) AS month,  DAY(datetime) AS day
FROM weather

GROUP BY MONTH(datetime), DAY(datetime)
HAVING MONTH(datetime) in (9)
ORDER BY month, day

'''

result = spark.sql(sql)
result.show()

+--------+--------+-----+---+
|ave_temp|max_temp|month|day|
+--------+--------+-----+---+
|   27.82|    30.8|    9|  1|
|    27.6|    30.7|    9|  2|
|   27.16|    29.9|    9|  3|
|   27.57|    30.9|    9|  4|
|   27.42|    30.8|    9|  5|
|   27.68|    31.0|    9|  6|
|   27.95|    32.1|    9|  7|
|   27.76|    31.9|    9|  8|
|   26.87|    31.1|    9|  9|
|   26.41|    30.9|    9| 10|
|   26.86|    30.0|    9| 11|
|   28.14|    31.8|    9| 12|
|   26.98|    30.0|    9| 13|
|   28.01|    30.7|    9| 14|
|   27.59|    29.9|    9| 15|
|   26.55|    28.9|    9| 16|
|   26.16|    28.9|    9| 17|
|   24.93|    28.0|    9| 18|
|   25.33|    29.0|    9| 19|
|   25.29|    28.9|    9| 20|
+--------+--------+-----+---+
only showing top 20 rows



In [ ]:
sql = '''
SELECT round(avg(temp),2) AS temp, YEAR(datetime) AS year, MONTH(datetime) AS month
FROM weather
GROUP BY YEAR(datetime), MONTH(datetime)
ORDER BY year, month

'''

result = spark.sql(sql)
result.show()

In [11]:
# average temperature by month and year

sql = '''
SELECT round(avg(temp),1) AS temp, YEAR(datetime) AS year, MONTH(datetime) AS month
FROM weather
GROUP BY YEAR(datetime), MONTH(datetime)
ORDER BY month, year

'''

result = spark.sql(sql)
result.show()

+----+----+-----+
|temp|year|month|
+----+----+-----+
|14.4|2024|    1|
|14.3|2025|    1|
|14.4|2024|    2|
|11.7|2025|    2|
|16.4|2024|    3|
|16.7|2025|    3|
|20.1|2024|    4|
|17.8|2025|    4|
|21.6|2024|    5|
|21.4|2025|    5|
|27.4|2024|    6|
|25.1|2025|    6|
|28.8|2024|    7|
|27.9|2025|    7|
|28.4|2024|    8|
|28.7|2025|    8|
|26.6|2024|    9|
|22.7|2024|   10|
|18.3|2024|   11|
|15.1|2024|   12|
+----+----+-----+



In [12]:
# average temperature particular month

sql = '''
SELECT YEAR(datetime) AS year, MONTH(datetime) AS month, round(AVG(temp),2) AS avg_temp
FROM weather
WHERE YEAR(datetime) IN (2025, 2024, 2023) AND MONTH(datetime) = 5
GROUP BY YEAR(datetime), MONTH(datetime);

'''

result = spark.sql(sql)
result.show()

+----+-----+--------+
|year|month|avg_temp|
+----+-----+--------+
|2025|    5|    21.4|
|2024|    5|   21.59|
+----+-----+--------+

